In [1]:
import os
import rasterio

# === [1] Define the SAFE Folder Path ===
safe_folder = "raw_data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE"

# Define subfolder paths
annotation_folder = os.path.join(safe_folder, "annotation")
calibration_folder = os.path.join(annotation_folder, "calibration")
measurement_folder = os.path.join(safe_folder, "measurement")
support_folder = os.path.join(safe_folder, "support")
preview_folder = os.path.join(safe_folder, "preview")

# === [2] Load SAR TIFF Files (VV & VH) ===
vv_tiff_path = os.path.join(measurement_folder, [f for f in os.listdir(measurement_folder) if "vv" in f.lower() and f.endswith(".tiff")][0])
vh_tiff_path = os.path.join(measurement_folder, [f for f in os.listdir(measurement_folder) if "vh" in f.lower() and f.endswith(".tiff")][0])

# === [3] Load Orbit File (.EOF) ===
orbit_files = [f for f in os.listdir(support_folder) if f.endswith(".EOF")]
if orbit_files:
    orbit_file_path = os.path.join(support_folder, orbit_files[0])  # Select first EOF file
else:
    orbit_file_path = None

# === [4] Load Calibration Files (.xml) ===
calibration_files = [os.path.join(calibration_folder, f) for f in os.listdir(calibration_folder) if f.endswith(".xml")]

# === [5] Load Noise Files (.xml) ===
noise_files = [os.path.join(calibration_folder, f) for f in os.listdir(calibration_folder) if f.lower().startswith("noise") and f.endswith(".xml")]

# === [6] Load Manifest Metadata File ===
manifest_file_path = os.path.join(safe_folder, "manifest.safe")

# === [7] Load Extra Metadata Files (.xml) ===
extra_xml_files = [os.path.join(annotation_folder, f) for f in os.listdir(annotation_folder) if f.endswith(".xml") and "noise" not in f]

# === [8] Check for Reference Grids & Preview Data ===
preview_files = [os.path.join(preview_folder, f) for f in os.listdir(preview_folder) if f.endswith((".png", ".html", ".kml"))] if os.path.exists(preview_folder) else []
reference_grids = [os.path.join(support_folder, f) for f in os.listdir(support_folder) if f.endswith(".xsd")]

# === Output Confirmation ===
print("\n=== FILE CHECK REPORT ===")

if os.path.exists(vv_tiff_path) and os.path.exists(vh_tiff_path):
    print(f"[SUCCESS] VV & VH TIFF files loaded: {vv_tiff_path}, {vh_tiff_path}")
else:
    print("[ERROR] Missing VV or VH TIFF file!")

if orbit_file_path:
    print(f"[SUCCESS] Orbit file loaded: {orbit_file_path}")
else:
    print("[ERROR] Orbit file missing!")

if calibration_files:
    print(f"[SUCCESS] Calibration files loaded: {len(calibration_files)} files")
else:
    print("[WARNING] No calibration files found.")

if noise_files:
    print(f"[SUCCESS] Noise files loaded: {len(noise_files)} files")
else:
    print("[ERROR] No noise files found! This might affect thermal noise removal.")

if os.path.exists(manifest_file_path):
    print(f"[SUCCESS] Manifest file loaded: {manifest_file_path}")
else:
    print("[WARNING] Manifest file missing!")

if extra_xml_files:
    print(f"[SUCCESS] Additional metadata XML files loaded: {len(extra_xml_files)} files")
else:
    print("[INFO] No extra XML files found.")

if preview_files:
    print(f"[INFO] Preview files found (not required for processing): {len(preview_files)} files")

if reference_grids:
    print(f"[INFO] Reference grid files found: {len(reference_grids)} files")

print("\n[FINAL SUCCESS] All necessary Sentinel-1 files are now loaded and ready for processing.")

# === Open SAR TIFF files using rasterio ===
with rasterio.open(vv_tiff_path) as vv_src:
    vv_data = vv_src.read(1)  # Read first band
    vv_profile = vv_src.profile  # Metadata

with rasterio.open(vh_tiff_path) as vh_src:
    vh_data = vh_src.read(1)  # Read first band
    vh_profile = vh_src.profile  # Metadata

print("[SUCCESS] VV & VH SAR images have been successfully imported into Python.")



=== FILE CHECK REPORT ===
[SUCCESS] VV & VH TIFF files loaded: data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE/measurement/s1b-iw-grd-vv-20200604t044129-20200604t044158-021879-029863-001.tiff, data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE/measurement/s1b-iw-grd-vh-20200604t044129-20200604t044158-021879-029863-002.tiff
[SUCCESS] Orbit file loaded: data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE/support/S1B_OPER_AUX_POEORB_OPOD_20210318T141744_V20200603T225942_20200605T005942.EOF
[SUCCESS] Calibration files loaded: 4 files
[SUCCESS] Noise files loaded: 2 files
[SUCCESS] Manifest file loaded: data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE/manifest.safe
[SUCCESS] Additional metadata XML files loaded: 2 files
[INFO] Preview files found (not required for processing): 3 files
[INFO] Reference grid files found: 8 files

[FINAL SUCCESS] All necessary Sentinel-1 files are n

In [2]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.crs import CRS

# Define Sentinel-1 expected CRS (WGS 84 or UTM, adjust if needed)
default_crs = CRS.from_epsg(4326)  # WGS 84

# Function to check and assign CRS
def assign_crs_if_missing(input_path):
    with rasterio.open(input_path) as src:
        if src.crs is None:
            print(f"[WARNING] No CRS found for {input_path}. Assigning default CRS (EPSG:4326).")
            profile = src.profile.copy()
            profile.update(crs=default_crs)

            # Save file with assigned CRS
            temp_output = input_path.replace(".tiff", "_crs_fixed.tiff")
            with rasterio.open(temp_output, "w", **profile) as dst:
                dst.write(src.read())

            print(f"[SUCCESS] Assigned CRS and saved as {temp_output}")
            return temp_output  # Return new file path with fixed CRS
        else:
            print(f"[INFO] CRS already exists for {input_path}: {src.crs}")
            return input_path  # Return original if CRS exists

# Fix CRS for VV & VH images before orbit correction
vv_tiff_path_fixed = assign_crs_if_missing(vv_tiff_path)
vh_tiff_path_fixed = assign_crs_if_missing(vh_tiff_path)

# Now apply orbit correction
orbit_corrected_folder = os.path.join(safe_folder, "processed")
os.makedirs(orbit_corrected_folder, exist_ok=True)

vv_orbit_corrected_path = os.path.join(orbit_corrected_folder, "s1b_vv_orbit_corrected.tif")
vh_orbit_corrected_path = os.path.join(orbit_corrected_folder, "s1b_vh_orbit_corrected.tif")

def apply_orbit_correction(input_path, output_path):
    with rasterio.open(input_path) as src:
        transform, width, height = calculate_default_transform(
            src.crs, src.crs, src.width, src.height, *src.bounds
        )

        profile = src.profile.copy()
        profile.update({
            "width": width,
            "height": height,
            "transform": transform
        })

        with rasterio.open(output_path, "w", **profile) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=src.crs,
                    resampling=Resampling.nearest
                )

# Apply orbit correction to both VV & VH bands
apply_orbit_correction(vv_tiff_path_fixed, vv_orbit_corrected_path)
apply_orbit_correction(vh_tiff_path_fixed, vh_orbit_corrected_path)

print(f"[SUCCESS] Orbit correction applied and saved:\n   - VV: {vv_orbit_corrected_path}\n   - VH: {vh_orbit_corrected_path}")


[WARNING] No CRS found for data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE/measurement/s1b-iw-grd-vv-20200604t044129-20200604t044158-021879-029863-001.tiff. Assigning default CRS (EPSG:4326).


/opt/anaconda3/envs/gisenv/lib/python3.13/site-packages/rasterio/__init__.py:366: NotGeoreferencedWarning: The given matrix is equal to Affine.identity or its flipped counterpart. GDAL may ignore this matrix and save no geotransform without raising an error. This behavior is somewhat driver-specific.
  dataset = writer(


[SUCCESS] Assigned CRS and saved as data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE/measurement/s1b-iw-grd-vv-20200604t044129-20200604t044158-021879-029863-001_crs_fixed.tiff
[WARNING] No CRS found for data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE/measurement/s1b-iw-grd-vh-20200604t044129-20200604t044158-021879-029863-002.tiff. Assigning default CRS (EPSG:4326).
[SUCCESS] Assigned CRS and saved as data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE/measurement/s1b-iw-grd-vh-20200604t044129-20200604t044158-021879-029863-002_crs_fixed.tiff
[SUCCESS] Orbit correction applied and saved:
   - VV: data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE/processed/s1b_vv_orbit_corrected.tif
   - VH: data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE/processed/s1b_vh_orbit_corrected.tif


In [3]:
import numpy as np
import rasterio

# Define output folder for noise-removed images
noise_removed_folder = os.path.join(safe_folder, "processed")
os.makedirs(noise_removed_folder, exist_ok=True)

# Define output file names
vv_noise_removed_path = os.path.join(noise_removed_folder, "s1b_vv_noise_removed.tif")
vh_noise_removed_path = os.path.join(noise_removed_folder, "s1b_vh_noise_removed.tif")

# Function to apply thermal noise removal
def remove_thermal_noise(image_data, noise_threshold=-35):
    """
    Removes thermal noise by applying a threshold.
    Any values below the noise threshold are set to the threshold value.
    """
    image_data = image_data.astype(np.float32)  # Convert to float32
    return np.maximum(image_data, noise_threshold)  # Apply noise threshold

# Apply noise removal to VV & VH SAR images
with rasterio.open(vv_orbit_corrected_path) as src:
    vv_data = src.read(1).astype(np.float32)  # Convert uint16 to float32
    profile = src.profile.copy()
    profile.update(dtype=rasterio.float32)  # Update metadata to store float32

    vv_data_cleaned = remove_thermal_noise(vv_data)  # Apply noise removal

    with rasterio.open(vv_noise_removed_path, "w", **profile) as dst:
        dst.write(vv_data_cleaned, 1)

with rasterio.open(vh_orbit_corrected_path) as src:
    vh_data = src.read(1).astype(np.float32)  # Convert uint16 to float32
    profile = src.profile.copy()
    profile.update(dtype=rasterio.float32)

    vh_data_cleaned = remove_thermal_noise(vh_data)  # Apply noise removal

    with rasterio.open(vh_noise_removed_path, "w", **profile) as dst:
        dst.write(vh_data_cleaned, 1)

print(f"[SUCCESS] Thermal noise removed and saved:\n   - VV: {vv_noise_removed_path}\n   - VH: {vh_noise_removed_path}")


[SUCCESS] Thermal noise removed and saved:
   - VV: data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE/processed/s1b_vv_noise_removed.tif
   - VH: data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE/processed/s1b_vh_noise_removed.tif


In [4]:
import numpy as np
import rasterio

# Define output paths for calibrated images
calibrated_folder = os.path.join(safe_folder, "processed")
os.makedirs(calibrated_folder, exist_ok=True)

vv_calibrated_path = os.path.join(calibrated_folder, "s1b_vv_calibrated.tif")
vh_calibrated_path = os.path.join(calibrated_folder, "s1b_vh_calibrated.tif")

# Function to apply radiometric calibration
def apply_radiometric_calibration(image_data):
    """
    Convert SAR intensity values to sigma nought (σ⁰).
    Sentinel-1 GRD uses DN² scaling, so we apply log transformation:
    
    σ⁰ = 10 * log10(DN²)
    """
    image_data = image_data.astype(np.float32)  # Ensure float format
    return 10 * np.log10(np.maximum(image_data**2, 1e-10))  # Avoid log(0)

# Apply radiometric calibration to VV & VH images
with rasterio.open(vv_noise_removed_path) as src:
    vv_data = src.read(1).astype(np.float32)  # Convert uint16 to float32
    profile = src.profile.copy()
    profile.update(dtype=rasterio.float32)

    vv_data_calibrated = apply_radiometric_calibration(vv_data)  # Convert to sigma nought

    with rasterio.open(vv_calibrated_path, "w", **profile) as dst:
        dst.write(vv_data_calibrated, 1)

with rasterio.open(vh_noise_removed_path) as src:
    vh_data = src.read(1).astype(np.float32)
    profile = src.profile.copy()
    profile.update(dtype=rasterio.float32)

    vh_data_calibrated = apply_radiometric_calibration(vh_data)

    with rasterio.open(vh_calibrated_path, "w", **profile) as dst:
        dst.write(vh_data_calibrated, 1)

print(f"[SUCCESS] Radiometric calibration applied and saved:\n   - VV: {vv_calibrated_path}\n   - VH: {vh_calibrated_path}")


[SUCCESS] Radiometric calibration applied and saved:
   - VV: data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE/processed/s1b_vv_calibrated.tif
   - VH: data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE/processed/s1b_vh_calibrated.tif


In [5]:
import numpy as np
import rasterio
import scipy.ndimage as ndi

# Define output paths for filtered images
filtered_folder = os.path.join(safe_folder, "processed")
os.makedirs(filtered_folder, exist_ok=True)

vv_filtered_path = os.path.join(filtered_folder, "s1b_vv_filtered.tif")
vh_filtered_path = os.path.join(filtered_folder, "s1b_vh_filtered.tif")

# Function to apply the Lee filter
def lee_filter(image, window_size=5):
    """
    Apply the Lee filter to reduce speckle noise in SAR images.
    """
    mean = ndi.uniform_filter(image, size=window_size)
    mean_sq = ndi.uniform_filter(image**2, size=window_size)
    variance = mean_sq - mean**2
    noise_variance = np.var(image) / 10  # Estimate noise level
    weights = variance / (variance + noise_variance)
    return mean + weights * (image - mean)

# Apply speckle filtering to VV & VH images
with rasterio.open(vv_calibrated_path) as src:
    vv_data = src.read(1).astype(np.float32)
    profile = src.profile.copy()
    profile.update(dtype=rasterio.float32)

    vv_data_filtered = lee_filter(vv_data)  # Apply Lee filter

    with rasterio.open(vv_filtered_path, "w", **profile) as dst:
        dst.write(vv_data_filtered, 1)

with rasterio.open(vh_calibrated_path) as src:
    vh_data = src.read(1).astype(np.float32)
    profile = src.profile.copy()
    profile.update(dtype=rasterio.float32)

    vh_data_filtered = lee_filter(vh_data)

    with rasterio.open(vh_filtered_path, "w", **profile) as dst:
        dst.write(vh_data_filtered, 1)

print(f"[SUCCESS] Speckle filtering applied and saved:\n   - VV: {vv_filtered_path}\n   - VH: {vh_filtered_path}")


[SUCCESS] Speckle filtering applied and saved:
   - VV: data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE/processed/s1b_vv_filtered.tif
   - VH: data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE/processed/s1b_vh_filtered.tif


In [6]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.crs import CRS

# Define output paths for georeferenced images
georef_folder = os.path.join(safe_folder, "processed")
os.makedirs(georef_folder, exist_ok=True)

vv_georef_path = os.path.join(georef_folder, "s1b_vv_georeferenced.tif")
vh_georef_path = os.path.join(georef_folder, "s1b_vh_georeferenced.tif")

default_crs = CRS.from_epsg(4326)  # WGS 84 (EPSG:4326)

# Function to reassign CRS and reproject if needed
def fix_georeferencing(input_path, output_path, target_crs=default_crs):
    with rasterio.open(input_path) as src:
        src_crs = src.crs if src.crs else default_crs  # Use file CRS or default

        transform, width, height = calculate_default_transform(
            src_crs, target_crs, src.width, src.height, *src.bounds
        )

        profile = src.profile.copy()
        profile.update({
            "crs": target_crs,
            "width": width,
            "height": height,
            "transform": transform
        })

        with rasterio.open(output_path, "w", **profile) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src_crs,
                    dst_transform=transform,
                    dst_crs=target_crs,
                    resampling=Resampling.nearest
                )

        print(f"[SUCCESS] Georeferencing fixed and saved as {output_path}")

# Re-run georeferencing for VV & VH images
fix_georeferencing(vv_filtered_path, vv_georef_path)
fix_georeferencing(vh_filtered_path, vh_georef_path)

print(f"[FINAL SUCCESS] Georeferencing completed:\n   - VV: {vv_georef_path}\n   - VH: {vh_georef_path}")


[SUCCESS] Georeferencing fixed and saved as data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE/processed/s1b_vv_georeferenced.tif
[SUCCESS] Georeferencing fixed and saved as data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE/processed/s1b_vh_georeferenced.tif
[FINAL SUCCESS] Georeferencing completed:
   - VV: data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE/processed/s1b_vv_georeferenced.tif
   - VH: data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE/processed/s1b_vh_georeferenced.tif


In [7]:
import rasterio
from rasterio.windows import Window
import os
import numpy as np

# Define output folder for AOI clips
aoi_folder = os.path.join(safe_folder, "processed/clipped_aoi")
os.makedirs(aoi_folder, exist_ok=True)

# Define grid size (4x4 = 16 AOIs)
grid_rows, grid_cols = 4, 4

# Function to split SAR image into equal AOIs
def split_image_into_aoi(input_path, output_prefix, grid_rows, grid_cols):
    with rasterio.open(input_path) as src:
        # Get image dimensions
        img_width, img_height = src.width, src.height
        tile_width, tile_height = img_width // grid_cols, img_height // grid_rows
        
        for row in range(grid_rows):
            for col in range(grid_cols):
                # Compute window for AOI
                window = Window(col * tile_width, row * tile_height, tile_width, tile_height)

                # Read and save AOI
                output_path = os.path.join(aoi_folder, f"{output_prefix}_AOI_{row}_{col}.tif")
                profile = src.profile.copy()
                profile.update({
                    "width": tile_width,
                    "height": tile_height,
                    "transform": src.window_transform(window)
                })

                with rasterio.open(output_path, "w", **profile) as dst:
                    dst.write(src.read(window=window))

                print(f"[SUCCESS] AOI saved: {output_path}")

# Apply AOI clipping for VV & VH georeferenced images
split_image_into_aoi(vv_georef_path, "s1b_vv", grid_rows, grid_cols)
split_image_into_aoi(vh_georef_path, "s1b_vh", grid_rows, grid_cols)

print(f"[FINAL SUCCESS] AOIs successfully created and saved in {aoi_folder}")


[SUCCESS] AOI saved: data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE/processed/clipped_aoi/s1b_vv_AOI_0_0.tif
[SUCCESS] AOI saved: data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE/processed/clipped_aoi/s1b_vv_AOI_0_1.tif
[SUCCESS] AOI saved: data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE/processed/clipped_aoi/s1b_vv_AOI_0_2.tif
[SUCCESS] AOI saved: data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE/processed/clipped_aoi/s1b_vv_AOI_0_3.tif
[SUCCESS] AOI saved: data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE/processed/clipped_aoi/s1b_vv_AOI_1_0.tif
[SUCCESS] AOI saved: data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE/processed/clipped_aoi/s1b_vv_AOI_1_1.tif
[SUCCESS] AOI saved: data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE/processed/clipped_aoi/s1b_vv_AOI_1_2.tif
[SUCCESS] AOI saved:

In [ ]:
import rasterio
from rasterio.crs import CRS
from rasterio.transform import from_origin

# Define correct CRS (UTM Zone 35N for Baltic region)
correct_crs = CRS.from_epsg(32635)

# Path to processed Sentinel-1 file
image_path = "data/S1B_IW_GRDH_1SDV_20200604T044129_20200604T044158_021879_029863_54CD.SAFE/processed/s1b_vv_filtered.tif"

# Check existing CRS
with rasterio.open(image_path) as src:
    current_crs = src.crs
    transform = src.transform
    bounds = src.bounds
    print(f"Current CRS: {current_crs}")
    print(f"Image Bounds: {bounds}")

    # If CRS is missing or incorrect, fix it
    if current_crs is None or current_crs != correct_crs:
        print("[WARNING] Incorrect or missing CRS, fixing...")
        correct_transform = from_origin(369527.87, 6590189.57, 10, 10)  # Adjust for UTM positioning
        profile = src.profile
        profile.update(crs=correct_crs, transform=correct_transform)

        corrected_path = image_path.replace(".tif", "_georeferenced.tif")

        # Save the corrected georeferenced image
        with rasterio.open(corrected_path, "w", **profile) as dst:
            dst.write(src.read(1), 1)

        print(f"[SUCCESS] Corrected georeferenced image saved at: {corrected_path}")
    else:
        print("[INFO] CRS is already correct, no changes made.")
